<a href="https://colab.research.google.com/github/rowntreerob/demo311/blob/master/Copy_of_photUp_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Point & Shoot pothole reports via AI
Version 2 of an AI model, trained for image classification in a [notebook format](https://www.dataquest.io/blog/jupyter-notebook-tutorial/) accessible to those interested in learning more about adapting media uploads to leverage AI and location in photos.
This version of the demo lets you convert 4 components from hosted services to YOUR OWN API accounts so you can take ownership of the OSS, using your own accounts and access keys to free storage services such as AWS cloud storage and a mongo database layer.

##Get started - Configure AWS, Maps, Roboflow, MongoDB

Instructions below on making 4 changes to the prior workbook code so that you can run the process using your own keys and your own accounts (aws, google, roboflow, mongoDB)

1. [aws docs](https://medium.com/@shamnad.p.s/how-to-create-an-s3-bucket-and-aws-access-key-id-and-secret-access-key-for-accessing-it-5653b6e54337) here cover creating your bucket and managing key information:
* access-key
* secret-access-key
* bucket-name

2. [maps docs](https://developers.google.com/maps/documentation/javascript/get-api-key) here cover signup and account creation:
* api-key

3. [AI on roboflow](https://docs.roboflow.com/api-reference/authentication#retrieve-an-api-key) here
* get your api-key
* Roboflow API - image classification cell, key in your api key value
* with your own account, [search here](https://universe.roboflow.com/search?q=classification%20graffiti&t=metadata) on **graffiti** and results with project **org311-photos** which you can clone.
  
4. [how to video](https://www.loom.com/share/bcdec71bf2a94cceba1ae2fc67be0606?sid=a5819317-2a47-41be-97a3-a346457d3361)  configure the python connection string to connect the cell to the database you create using the mongodb dashboard

##A sample Photo
 Here is a graffiti photo, including the street number (2933). Its your choice on the photo input. Either upload the photo included here or select and upload your own photo for classification by the demo.
![sample 1](https://awsgcpupld-production.up.railway.app/pics/rclass_1.jpg)

##Instructions to complete this excercise
1. download this photo - right clik on it then "save image as.. " OR use your own photo
2. run the cell installing EXIF and other components
3. run cell to configure AWS, various components with respective keys
4. UPLOAD to colab, the photo just downloaded by running cell "UPLOAD"
5. verify size of upload -> 3 gig size constraint for Roboflow step
6. run the 2 cells below that to get latitude , longitude and Address
7. run image classification cell to get the type of photo and issue type
8. run the cell to insert into DB the issue and data

##Note on running a cell
Cells in Colab notebooks have icons in the far upper left of the cell. A mouseover on each icon will display details of instructions on the controls that you use within cells in a notebook. Once selected, runnable cells have a grey triangle icon in small black circle. Clik it to run a cell. When complete, observe the bottom row of the cell where output of the cell appears.

![run a cell](https://awsgcpupld-production.up.railway.app/pics/runColab.png)

##Install EXIF module

In [ ]:
# run this cell one-time to install software in CoLab sandbox
!pip install exif
!pip install pymongo
!pip install awscli
!pip install roboflow
!pip install colab-env -qU

## Configuration
Note - you will need a google account for this step and you should be familiar with access on google drive where your configuration values are stored for access by this notebook cell. [detls on config for colab](https://colab.research.google.com/github/apolitical/colab-env/blob/master/colab_env_testbed.ipynb)  
This step sets values for the various services that support Maps, AI, storage and database particular for your version of the project.
Enter your values for the various keys, replacing both the angle brackets and the enclosed **your key** that you see below.  
  
sample config:  
"AWS_SECRET_ACCESS_KEY", "gGh4...6lDli",   
"AWS_BUCKET", "media",  
"MAPS_KEY", "AIz...dg5_LA",  
"ROBOFLOW_KEY", "1f7...twN",  
"DB_AUTH", "joe:H7BvLy17",  

**Get Started** section above has details

In [ ]:
import colab_env
colab_env.envvar_handler.add_env("AWS_ACCESS_KEY_ID", "<your key>", overwrite=True)
colab_env.envvar_handler.add_env("AWS_SECRET_ACCESS_KEY", "<your secret-key>", overwrite=True)
colab_env.envvar_handler.add_env("AWS_BUCKET", "<your bucket-name>", overwrite=True)
colab_env.envvar_handler.add_env("MAPS_KEY", "<your maps-key>", overwrite=True)
colab_env.envvar_handler.add_env("ROBOFLOW_KEY", "<your api-key>", overwrite=True)
colab_env.envvar_handler.add_env("DB_AUTH", "<user>:<password>", overwrite=True)
!more gdrive/My\ Drive/vars.env


##Select a photo for input in cells below
Run the cell, then use button "choose files" to open a "chooser" in your browser.

Note - this cell is the beginning of each iteration of the notebook.Start with either a photo of yours OR with the sample photo above (graffiti).   Process a number of your own photos thru the steps of the notebook (this cell thru "Database insert" cell). Each time your repeat the process, another record of an issue will insert to the Database. After you have cycled thru a handful of your photos, the last 2 cells in the notebook lookup a record from the DB, using that record to encode location for use in the last cell for a query to the DB creating a map of the issues in the query result set.

In [ ]:
import os
from google.colab import files
nwname = None
uploaded = files.upload()

for fn in uploaded.keys():
  fname = fn
  y=len(uploaded[fn])
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

if y>2950000: print('downscale this image before its classified using downscale cell')

Saving qa_3.jpg to qa_3.jpg
User uploaded file "qa_3.jpg" with length 1284716 bytes


In [ ]:
from PIL import Image

if y>2950000:
  print('file downsize image ')
  name,ext = os.path.splitext(fname)
  nwname = name + "-sm" + ext
  image = Image.open(fname)
  imgsml = image.resize((1000, 800))
  imgsml.save(nwname)

##Cloud Storage for photo on AWS S3

In [ ]:
# EDIT - 'AWS bucket name' from AWS process AND from configuration step
BUCKET = os.environ['AWS_BUCKET']
# no changes needed in code below
!aws s3 cp ./{fname} s3://{BUCKET} --acl public-read
s3url = f'https://{BUCKET}.s3.amazonaws.com/{fname}'
print(f"url: {s3url}")



upload: ./qa_3.jpg to s3://media-t/qa_3.jpg                       
url: https://media-t.s3.amazonaws.com/qa_3.jpg


##Interpret the Photo's GPS
metadata of the photo includes latitude/ longitude that can be read using "EXIF" software. Prints out values for lat long.

In [ ]:
from exif import Image
# using the uploaded image above as input
# call  gps parse functions for latitude / longitude from image data
# 2 functions defined below are used by stmt at the bottom "image_coordinates"
def decimal_coords(coords, ref):
 decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
 if ref == 'S' or ref == 'W':
     decimal_degrees = -decimal_degrees
 return decimal_degrees

def image_coordinates(img_path):
    coords = (0,0)
    with open(img_path, 'rb') as src:
        img = Image(src)
    if img.has_exif:
        try:
            img.gps_longitude
            coords = (decimal_coords(img.gps_latitude,
                      img.gps_latitude_ref),
                      decimal_coords(img.gps_longitude,
                      img.gps_longitude_ref))
        except AttributeError:
            print ('No Coordinates')
    else:
        print ('The Image has no EXIF information')
    return coords

# Step 1:  file from above to funtion that gets gps
respGps = image_coordinates(fname)
# format the response into variables ref'd by cell below
lat,long = respGps
print(f"lat: {lat}, long: {long}")


lat: 37.762147222222225, long: -122.41539444444444


##Google maps addresses
 the 2 following cells to get a proper street address from the lat/ long coordinate values above.

## Using your own maps account
You may plug in your [google maps api key](https://developers.google.com/maps/documentation/embed/get-api-key) using the cell below to get street address from coordinates (latitude, longitude)

In [ ]:
#get st. address from gps coordinates using your api-key
import os
import requests
# EDIT - your google maps API key
# KEY = 'AI' # replace whats inside the single quote with your key
KEY = os.environ['MAPS_KEY']
# Step 2: Construct the request and make the request
url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{long}&key={KEY}'
resp = requests.get(url).json()
coordinates = [long, lat]
address = resp["results"][0]["formatted_address"]
print(resp["results"][0]["formatted_address"])
print(f'address from coordinates: {long:.5f} {lat:.5f}')


3214-3216 18th St, San Francisco, CA 94110, USA
address from coordinates: -122.41539 37.76215


##Roboflow API - image classification (gargage, graffiti, or encampment)

```
from roboflow import Roboflow

rf = Roboflow(api_key={API_KEY})
project = rf.workspace().project("org311-clip-photos")
model = project.version(2).model

# infer on a local image
print(model.predict("rclass_1.jpg").json())
```
The sample python code above from Roboflow samples performs a request for classification according to the trained, **org311-clip** model providing answer ( mural, garbage, encampment, graffiti ). This is the AI supplying the issue-type after analysis of the photo.

####Security Note

**API_KEY** in order to secure the key value, for the purpose of this demo, the code above is wrapped in a proxy. The actual call you will run in the cell below calls the proxy rather than directly creating a project and a model to execute the call.

Using a single line of code, the proxy securely appends the key, calling a REST endpoint for the image classification by Roboflow model.
```
await axios.post(`https://classify.roboflow.com/org311-clip-photos/2?api_key=${Config.api.ROBOFLOWKEY}`
```



In [ ]:
from roboflow import Roboflow

# EDIT - copy (3) lines in from app.roboflow.com deploy model and tab for python
KEY = os.environ['ROBOFLOW_KEY']
rf = Roboflow(KEY)
# rf = Roboflow(api_key="1fN")
project = rf.workspace().project("org311-clip-photos")
model = project.version(2).model
# fn2 = "smaller.jpeg"

# infer on a local image get 2 predicts
if nwname == None:
  resp = model.predict(fname).json()["predictions"][0]["predictions"]
else:
  resp = model.predict(nwname).json()["predictions"][0]["predictions"]

# print out the first 2 predictions for type of photo
predictions = [resp[0],resp[1]]
print(predictions)


loading Roboflow workspace...
loading Roboflow project...
[{'class': 'encampment', 'confidence': 0.6519}, {'class': 'garbage', 'confidence': 0.1361}]


##Database storage - insert issue in DB
This notebook assumes you have created your own database instance.
Proper configuration must be set with your connection variables set in 2 places

- Configuration cell above **user , password**
- uri value below as explained in [video clip](https://www.loom.com/share/bcdec71bf2a94cceba1ae2fc67be0606)

With DB insert, the process has been completed on a photo and with the completion of this cell, you can return to the steps above, beginning with selecting a new photo. With 5 or more DB inserts complete, you can leave the interative process above and complete the following cells (2) to place the issue locations on a map.

In [ ]:
from pymongo import MongoClient
from datetime import datetime
import os
# connection string for mongodb
# see videoclip for edit on the uri string used to connect to DB : DB_AUTH
AUTH = os.environ['DB_AUTH']
uri = f'mongodb+srv://{AUTH}@org311.vo6xmd4.mongodb.net/?retryWrites=true&w=majority'
client = MongoClient(uri)
db = client["org311"]
issue = {"location": {
      "type": "Point",
      "coordinates": coordinates
  }, "address": address,
  "predictions":predictions,
  "url": s3url,
  "last_modified": datetime.utcnow()
}
issues = db.issues
issueId = issues.insert_one(issue).inserted_id
print(issueId)
#issueId required below

64ecc3c73a89ffec5fc04fb0


In [ ]:

from pymongo import MongoClient
from datetime import datetime
from bson import ObjectId
import os

#pull in issueId from prev cell's insert to DB
myId = issueId
objInstance = ObjectId(myId)

# connection string for mongodb
# see videoclip for edit on the uri string used to connect to DB : DB_AUTH
AUTH = os.environ['DB_AUTH']
uri = f'mongodb+srv://{AUTH}@org311.vo6xmd4.mongodb.net/?retryWrites=true&w=majority'
client = MongoClient(uri)
db = client["org311"]
collection = db.issues
result = collection.find_one({'_id': objInstance},
                               {'_id': 0,  'location.coordinates': 1,  'predictions' : 1})
print(result["location"])
rsltLocation = result["location"]["coordinates"]


{'coordinates': [-122.41539444444444, 37.762147222222225]}


##Populate DB. Query the DB and Map results.
Cycle thru the notebook's steps, each time submitting new photos of your own, populating the DB with your issues and the respective locations of each issue ( processed by the notebook and inserted to the DB layer ).
Use this cell to produce a result set of points within 700 meters of a **near** point whose geometry you edit. Any issues in the DB within that distance of the point you set will be mapped with the URL of the map page appearing as output in the cell. Run cell and click on the link which is output to see your map.

Details - DB query to find "location near geometry" documentation [here](https://www.mongodb.com/docs/manual/geospatial-queries/)

Details - [setting URL](https://developers.google.com/maps/documentation/maps-static/start) value prior to calling Maps API

In [ ]:
from pymongo import MongoClient
from datetime import datetime
import os
import json
import urllib.parse
# connection string for mongodb
# see videoclip for edit on the uri string used to connect to DB
AUTH = os.environ['DB_AUTH']
MAPS = os.environ['MAPS_KEY']
uri = f'mongodb+srv://{AUTH}@org311.vo6xmd4.mongodb.net/?retryWrites=true&w=majority'
client = MongoClient(uri)
db = client["org311"]
col = db["issues"]
collection = db['test_collection']
#use geo Index to find DB entries near a point ( 700 meters )
#returns a results set of nearby issues
cursor = col.find(
 {
   "location": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": rsltLocation
       },
       "$maxDistance": 700
     }
   }
},{"_id": 0, "location.coordinates": 1, "address": 1, "predictions[0]class": 1}
)
# section below req'd to set value of URL for a call on static maps API
list = ''
for i, rec in enumerate(cursor):
  #use lat long from results set issues to create markers for a map
  l = rec["location"]["coordinates"][1]
  r = rec["location"]["coordinates"][0]
  ele = f'{l},{r}'
  if i > 0:
    list = list + '|'
  list = list + ele
#use maps api to create a map of issues
marker = f'color:red|label:S|{list}'
encode = urllib.parse.quote_plus(marker)
url = f'https://maps.googleapis.com/maps/api/staticmap?center={l},{r}&zoom=15&size=640x640&markers={encode}&key={MAPS}'
print(url)

https://maps.googleapis.com/maps/api/staticmap?center=37.763977777777775,-122.40851944444445&zoom=15&size=640x640&markers=color%3Ared%7Clabel%3AS%7C37.762147222222225%2C-122.41539444444444%7C37.762147222222225%2C-122.41539444444444%7C37.762147222222225%2C-122.41539444444444%7C37.762147222222225%2C-122.41539444444444%7C37.76173333333333%2C-122.41590833333333%7C37.76164444444444%2C-122.41465833333334%7C37.76164444444444%2C-122.41465833333334%7C37.761069444444445%2C-122.41584166666667%7C37.76047222222222%2C-122.41578333333334%7C37.76050277777778%2C-122.41421944444446%7C37.76390277777778%2C-122.41370277777779%7C37.76401111111111%2C-122.41298611111111%7C37.75904444444444%2C-122.41466944444446%7C37.75904444444444%2C-122.41466944444446%7C37.75893055555556%2C-122.41574722222222%7C37.76495%2C-122.41756111111111%7C37.757266666666666%2C-122.41516111111112%7C37.75714722222222%2C-122.41558888888889%7C37.76345555555555%2C-122.40871666666668%7C37.76345555555555%2C-122.40871666666668%7C37.763455555555